In [217]:
import sys
import itertools
disjunction = lambda *args: any(args)

In [245]:
def find_pure(clauses):
    pure = {}
    reached = []
    for clause in clauses:
        for lit in clause:
            if lit[0]=='~':
                if lit[1:] in pure.keys():
                    if pure[lit[1:]]!=False:
                        del pure[lit[1:]]
                elif lit[1:] not in reached:
                    pure[lit[1:]]=False
                    reached.append(lit)
            else:
                if lit in pure.keys():
                    if pure[lit]!=True:
                        del pure[lit]
                elif lit not in reached:
                    pure[lit]=True
                    reached.append(lit)
    
    #add the clause index where index = find(pure)
    '''listOfRemovals = []
    for j in range(len(clauses)):'''

    
    return pure

_________


In [314]:
def unit_heuristic(clauses, assignment):
    clauses2 = clauses.copy()
    for i in range(len(clauses2)):
        clause = clauses2[i]
        if len(clause)==1:
            c = clause[0] 
            if c[0]=="~":
                #add to assignment if it does not exist, if it exists, then it must be from contradicting unit clauses and it is unsat
                if c[1:] in assignment and assignment[c[1:]]!=False:
                    #stands for unsatisfiable
                    return None, None 
                elif c[1:] not in assignment:
                    assignment[c[1:]] = False     
            
            else:
                if c in assignment and assignment[c]!=True:
                    return None, None #stands for unsatisfiable
                elif c not in assignment:
                    assignment[c] = True
            
            #remove clause
            clauses2[i] = None
    
    updated_clauses = [i for i in clauses2 if i is not None]
                    
    return assignment, updated_clauses

-------

In [247]:
#extract literals, clauses and pure literals

def extract(lines):
    literals = set()
    pure = dict() #assignments to pure
    clauses = []
    clause_idx = 0
    
    for line in lines:
    #each clause is a disjunction in cnf form
        clause = []
        for c in line.split(','):
            literals.add(c if c[0]!='~' else c[1:]) #add positive literal only
            clause.append(c)
        clauses.append(clause)

    return list(literals), clauses

In [207]:
lines = ['A,B', 'C,D', "~A","~B"]
literals, clauses = extract(lines)

In [208]:
literals

['D', 'A', 'C', 'B']

In [209]:
clauses

[['A', 'B'], ['C', 'D'], ['~A'], ['~B']]

In [248]:

#given an assignment, check if assignment returns True for the clauses
def isSat(clauses, assignment):
    for c in clauses:
        clause = []
        for literal in c:
            if literal[0]=='~':
                clause.append(not assignment[literal[1:]])
            else: 
                clause.append(assignment[literal])             
        #if any disjunction is False, the entire CNF expression will be false
        if disjunction(*clause) is False:
            return False
    return True


In [309]:
def DPLL(clauses, literals, model):
    if len(literals)==0: #all assigned
        #find if model is satisfiable with current clauses #satisfiable should return True or False
        if isSat(clauses, model):
            return model
        else:
            return None 
    
    unit_assign, updated_clauses = unit_heuristic(clauses, model)
    if unit_assign is None:
        return None
    
    pure = find_pure(updated_clauses)

    #print(unit_assign)
    d3 = {}
    d3.update(pure)
    #print("after pure", d3)
    d3.update(unit_assign)
    #print("after unit", d3)
    d3.update(model)
    #print("after model", d3)


    if len(d3)==len(literals):
        if isSat(clauses,d3):
            return d3
        else:
            return None

    rest = [i for i in literals if i not in d3.keys()] #unassigned_keys
    first = rest.pop()

    
    return DPLL(updated_clauses, rest, model ={**d3, **{first:True}} ) or DPLL(updated_clauses, rest, model = {**d3, **{first:False}})


    


In [313]:
print(DPLL(clauses, literals, {}))

{'A': True, 'B': True, 'C': False, 'D': True}


In [312]:
lines = ['~A,B', '~C,D', "A","B"]
literals, clauses = extract(lines)

In [216]:

#perform SAT with values
inputLines = sys.stdin.read().splitlines()
lit, clauses = literals(inputLines)


KeyboardInterrupt: 

----

IGNORE
_______

In [214]:

#find all possible combination of T/F values with Cartesian Product
def assign(literals):
    n = len(literals)
    for i in itertools.product([True,False], repeat=n):
        yield (dict(zip(literals, i)))

In [215]:
#main solver
def cnf(lit, clauses):
    for assignment in assign(lit):
        if isSat(clauses, assignment) is True:
            print("satisfiable", end = " ")
            for key in assignment.keys():
                print(key+'='+str(assignment.get(key))[0],end=' ')
            print()
            return
    print('unsatisfiable')
    return
